数据载入、存储及文件格式

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_note_interactivity = "all"
%matplotlib inline

import os
import sys
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

pandas中的文件解析函数：

read_csv***

read_table***

read_excel***

read_fwf

read_clipboard

read_hdf

read_html

read_json***

read_msgpack

readk_pickle

read_sas

read_sql

read_stata

read_feather

*******指最为常用的解析函数

In [2]:
os.getcwd()

'E:\\py_pandasDA_practice'

In [3]:
os.chdir('E:\\py_pandasDA_practice\\code')#换一下路径

In [4]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [5]:
df = pd.read_csv('examples/ex1.csv')

In [6]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [9]:
#控制pandas在读取时是否自动分配默认列名

pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
#读取时设置某列为索引

names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


需要从多格列中形成分层索引的话，需要传入一个包含列序号或列名的列表：

In [12]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                    index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

在一张表的分隔符并不固定的情况下，使用空白或其他方式来分隔字段，考虑如下案例：

In [14]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [15]:
#可以使用read_table并传入一个正则表达式作为分隔符。

result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [16]:
#解析函数提供了许多的附加参数帮助用户处理各种异常的文件格式。

!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [17]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [18]:
#缺失值的处理

!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [19]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [21]:
result = pd.read_csv('examples/ex5.csv', na_values=['one'])
result

,something,a,b,c,d,message
0,NaN,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
#利用字典标定不同列的缺失值标识

sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


遇到大文件时可以分块导入，或按照小块遍历文件

In [23]:
#紧凑化显示设置
pd.options.display.max_rows = 10

In [24]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [25]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [26]:
#利用chunksize来限制每一块的行数
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [27]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [28]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [29]:
data.to_csv('examples/out.csv')
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [30]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [31]:
#使用其他标识值给缺失值做标记
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [32]:
data.to_csv(sys.stdout, index=False, header=False)#取消写入索引和列名

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [33]:
#写入列自己，并按照用户选择的顺序进行写入
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [34]:
#Series同样可以to_csv

dates = pd.date_range('1/1/2000', periods=7)

In [35]:
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


JSON的读取与写入比较常见，因为它是一种主流的互联网数据交换形式

In [36]:
obj = """
{"name": "Wes",
"places_lived": ["US", "Spain", "Germany"],
"pet": null,
"siblings": [{"names": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
            {"names": "Katie", "age": 38,
            "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [37]:
import json

In [38]:
#将JSON字符串转换为Python形式
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['US', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'names': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'names': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [39]:
asjson = json.dumps(result)

In [40]:
type(asjson)

str

In [41]:
asjson

'{"siblings": [{"age": 30, "names": "Scott", "pets": ["Zeus", "Zuko"]}, {"age": 38, "names": "Katie", "pets": ["Sixes", "Stache", "Cisco"]}], "pet": null, "name": "Wes", "places_lived": ["US", "Spain", "Germany"]}'

In [42]:
siblings= pd.DataFrame(result['siblings'], columns=['names', 'age'])
siblings

,names,age
0,Scott,30
1,Katie,38


In [43]:
# pandas.read_json可以自动将JSON数据集按照指定次序转换为Series或DF
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [44]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [45]:
# to_json
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [46]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


HTML/XML

In [47]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [48]:
len(tables)

1

In [49]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [50]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

In [51]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root= parsed.getroot()

In [52]:
data= []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
perf = pd.DataFrame(data)

In [53]:
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [54]:
from io import StringIO

tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [55]:
root

<Element a at 0x266605bec88>

In [56]:
root.get('href')

'http://www.google.com'

In [57]:
root.text

'Google'

二进制格式

to_pickle

read_pickle

In [58]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [59]:
frame.to_pickle('examples/frame_pickle')

In [60]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [61]:
type(pd.read_pickle('examples/frame_pickle'))

pandas.core.frame.DataFrame

HDF5是一种用于存储大量科学数组的数据结构，适合一次存，多次取（注意：反复写入可能会导致文件挂掉）

PyTables and h5py

HDF5适合处理不适合在内存中存储的超大型数据，可以使用户高效读写大型数组的一个小块。

支持多种模式压缩。

“HDF”代表分层数据格式，每个HDF5文件可以存储多个数据集并支持元数据。

In [62]:
#pandas中的HDFStore
frame = pd.DataFrame({'a': np.random.randn(100)})

In [63]:
frame

,a
0,-0.160953
1,-0.450246
2,-0.513633
3,1.508734
4,-0.095021
...,...
95,-0.407136
96,0.385956
97,1.133184
98,0.446073


In [64]:
store = pd.HDFStore('mydata.h5')

In [65]:
store['obj1'] = frame

In [66]:
store['obj1_col'] = frame['a']

In [67]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[100,1])
/obj1_col            series       (shape->[100])  

Microsoft Excel文件

In [68]:
#这种模式适合excel文件中有多个表的情况，会比较快
xlsx = pd.ExcelFile('examples/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [69]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [70]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [71]:
frame.to_excel('examples/ex2.xlsx')

API

In [72]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [73]:
resp = requests.get(url)
resp

<Response [200]>

In [74]:
data = resp.json()
data[0]['title']

'Fix type annotation in pandas.compat'

In [75]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [76]:
issues

,number,title,labels,state
0,26252,Fix type annotation in pandas.compat,"[{'default': False, 'node_id': 'MDU6TGFiZWwxMj...",open
1,26251,Fix Type Annotation in pandas.compat,"[{'default': False, 'node_id': 'MDU6TGFiZWwxMj...",open
2,26250,DOC removed unnecessary use of parameter in ex...,"[{'default': False, 'node_id': 'MDU6TGFiZWwxMz...",open
3,26249,Some cleanups of merge_asof tests and error code,"[{'default': False, 'node_id': 'MDU6TGFiZWwxMz...",open
4,26247,numpy ravel does not work on a list of DataFra...,[],open
...,...,...,...,...
25,26210,BUG: Sorting MultiIndex with NaNs,"[{'default': False, 'node_id': 'MDU6TGFiZWw0Mj...",open
26,26209,reindex does not work for groupby series with ...,"[{'default': False, 'node_id': 'MDU6TGFiZWw3Nj...",open
27,26208,SeriesGroupBy.transform cannot handle empty se...,"[{'default': False, 'node_id': 'MDU6TGFiZWwyMz...",open
28,26206,REGR? no error anymore when converting out of ...,"[{'default': False, 'node_id': 'MDU6TGFiZWwyMT...",open


数据库

In [77]:
import sqlite3

In [78]:
query = """
CREATE TABLE test2
(a VARCHAR(20), b VARCHAR(20),
c real, d INTEGER
);
"""

In [79]:
con = sqlite3.connect('mydata.sqlite')

In [80]:
con.execute(query)

In [81]:
con.commit()

In [82]:
#插入数据示例
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [83]:
stmt = "INSERT INTO test2 VALUES(?, ?, ?, ?)"

In [84]:
con.executemany(stmt, data)

In [85]:
#从数据库中的表中选择数据（大部分python的SQL驱动（PyODBC/psycopg2/MySQLdb/pymssql）返回的是元组的列表）

In [86]:
cursor = con.execute('select * from test2')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [87]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [88]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


扩展阅读SQLAlchemy

In [89]:
import sqlalchemy as sqla

In [90]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [91]:
pd.read_sql('select * from test2', db)

,a,b,c,d
